In [77]:
import pandas as pd
import random
import os
import numpy as np
import matplotlib.pyplot as plt
import warnings; warnings.filterwarnings(action='ignore')
import pickle
import time
from sklearn.ensemble import GradientBoostingClassifier
from scipy.stats import randint
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from bayes_opt import BayesianOptimization
from xgboost import XGBClassifier # model 
from xgboost import plot_importance
from sklearn.ensemble import VotingClassifier
from lightgbm import LGBMClassifier
from scipy.stats import randint
import sklearn.svm as svm


from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor

In [78]:
import sys   
!{sys.executable} -m pip install time

ERROR: Could not find a version that satisfies the requirement time (from versions: none)
ERROR: No matching distribution found for time


In [79]:
def seed_everything(seed):
    random.seed(seed) #파이썬 자체 모듈 random 모듈의 시드 고정
    os.environ['PYTHONHASHSEED'] = str(seed) 
    np.random.seed(seed) #넘파이를 사용할 경우
#     torch.manual_seed(seed)
#     torch.cuda.manual_seed(seed)
#     torch.backends.cudnn.deterministic = True
    
seed_everything(37) # Seed 고정

# Data Load

In [80]:
#데이터프레임 불러오기
train_df = pd.read_csv('./train.csv')
test_df = pd.read_csv('./test.csv')

In [55]:
sub1 = pd.read_csv('./sample_submission.csv')
sub2 = pd.read_csv('./sample_submission.csv')

In [73]:
sub3= pd.read_csv('./sample_submission.csv')

In [81]:
sub4= pd.read_csv('./sample_submission.csv')

In [97]:
sub5 = pd.read_csv('./sample_submission.csv')
sub6 = pd.read_csv('./sample_submission.csv')

# Data Preprocessing

PRODUCT_CODE 데이터 나누기

1.train_df

In [86]:
train_dfa = train_df[train_df['PRODUCT_CODE']=='A_31'] #249
train_dft = train_df[train_df['PRODUCT_CODE']=='T_31'] #343
train_dfo = train_df[train_df['PRODUCT_CODE']=='O_31'] #6

In [98]:
train_dfa['X_1228'][(train_dfa['Y_Class']==1) & (train_dfa['LINE']=='T050304')].mean()

73.02702702702703

In [87]:
#PRODUCT_CODE 별로 이 함수 이용
def fill_nan(df): #미디언으로 해주는 이유는 yclass가 0인건 55 1인건 59 이런 식으로 규칙이 존재하기 때문에 함부로 평균을 내면 안될 것 같음
    for col in df.columns[-2875:]: #X_1 ~ X_1531 , -1345
        #모두 결측치 인 경우 채우지 않는다.
        #T050 라인만 측정된 경우, 냅두기
        #T010 라인만 측정된 경우, 냅두기

        if df[col].isnull().sum() == len(df): #모든 value가 결측치인 행 null 249개, 343개, 6개
            continue
            
        elif len(df) - df[col].isnull().sum() <= 10: #너무 적은 개수만큼 측정되었다면, 그냥 채우지 않기 train_df_o의 경우 x
            continue
            
        elif df[col].isnull().sum() == 0: #모든 값이 다 측정된 경우 null 0개
            continue
            
        elif df[col].isnull().sum() < len(df): #T050이나 T010라인만 측정된 경우 어차피 값이 들어가지 않을 것임.
            nan_lst = list(df[df[col].isnull()==True].index)
            print(f'{col}')
            for j in nan_lst:
                yc = df['Y_Class'][j]
                line = df['LINE'][j]
                val = df[col][(df['Y_Class']==yc) & (df['LINE']==line)].median()
                df[col][j] = val
#                 print(f'df[col][j]에 들어가는 값은 {df[col][j]}')
        
    return df

In [88]:
start = time.time()
train_dfaf = fill_nan(train_dfa)
train_dftf = fill_nan(train_dft)
end = time.time()
print(f'time elapsed: {end-start}')

X_128
X_129
X_130
X_131
X_132
X_133
X_134
X_135
X_136
X_137
X_138
X_139
X_140
X_141
X_142
X_143
X_144
X_145
X_146
X_147
X_148
X_149
X_150
X_151
X_152
X_153
X_154
X_155
X_156
X_157
X_158
X_159
X_160
X_161
X_162
X_163
X_164
X_165
X_166
X_167
X_168
X_169
X_170
X_171
X_172
X_173
X_174
X_175
X_176
X_177
X_178
X_179
X_180
X_181
X_182
X_183
X_184
X_185
X_186
X_187
X_188
X_189
X_190
X_191
X_192
X_193
X_194
X_195
X_196
X_197
X_198
X_199
X_200
X_201
X_202
X_203
X_204
X_205
X_206
X_207
X_208
X_209
X_210
X_211
X_212
X_213
X_214
X_215
X_216
X_217
X_218
X_219
X_220
X_221
X_222
X_223
X_224
X_225
X_226
X_227
X_228
X_229
X_230
X_231
X_232
X_233
X_234
X_235
X_236
X_237
X_238
X_239
X_240
X_241
X_242
X_243
X_244
X_245
X_246
X_247
X_248
X_249
X_250
X_251
X_252
X_253
X_254
X_255
X_256
X_257
X_258
X_259
X_260
X_261
X_262
X_263
X_264
X_265
X_266
X_267
X_268
X_269
X_270
X_271
X_272
X_273
X_274
X_275
X_276
X_277
X_278
X_279
X_280
X_281
X_282
X_283
X_285
X_286
X_287
X_288
X_289
X_290
X_291
X_292
X_293
X_294
X_29

X_2126
X_2127
X_2128
X_2129
X_2130
X_2131
X_2132
X_2133
X_2134
X_2135
X_2136
X_2137
X_2138
X_2139
X_2140
X_2141
X_2142
X_2143
X_2144
X_2145
X_2146
X_2147
X_2148
X_2149
X_2150
X_2151
X_2152
X_2153
X_2154
X_2155
X_2156
X_2157
X_2158
X_2159
X_2160
X_2161
X_2162
X_2163
X_2164
X_2165
X_2166
X_2167
X_2168
X_2169
X_2170
X_2171
X_2172
X_2173
X_2174
X_2175
X_2176
X_2177
X_2178
X_2179
X_2180
X_2181
X_2182
X_2183
X_2184
X_2185
X_2186
X_2187
X_2188
X_2189
X_2190
X_2191
X_2192
X_2193
X_2194
X_2195
X_2196
X_2197
X_2198
X_2199
X_2200
X_2201
X_2202
X_2203
X_2204
X_2205
X_2206
X_2207
X_2208
X_2209
X_2210
X_2211
X_2212
X_2213
X_2214
X_2215
X_2216
X_2217
X_2218
X_2219
X_2220
X_2221
X_2222
X_2223
X_2224
X_2225
X_2226
X_2227
X_2228
X_2229
X_2230
X_2231
X_2232
X_2233
X_2234
X_2235
X_2236
X_2237
X_2238
X_2239
X_2240
X_2241
X_2242
X_2243
X_2244
X_2245
X_2246
X_2247
X_2248
X_2249
X_2250
X_2251
X_2252
X_2253
X_2254
X_2255
X_2256
X_2257
X_2258
X_2259
X_2260
X_2261
X_2262
X_2263
X_2264
X_2265
X_2266
X_2267
X_2268

In [89]:
train_dfof = fill_nan(train_dfo)
train_dfof

,PRODUCT_ID,Y_Class,Y_Quality,TIMESTAMP,LINE,PRODUCT_CODE,X_1,X_2,X_3,X_4,...,X_2866,X_2867,X_2868,X_2869,X_2870,X_2871,X_2872,X_2873,X_2874,X_2875
569,TRAIN_569,1,0.530533,2022-09-03 18:32,T100304,O_31,4.0,98.0,0.0,45.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
570,TRAIN_570,2,0.534951,2022-09-03 18:40,T100306,O_31,6.0,90.0,0.0,45.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
571,TRAIN_571,1,0.525916,2022-09-03 18:48,T100304,O_31,4.0,100.0,0.0,45.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
572,TRAIN_572,2,0.535205,2022-09-03 18:56,T100306,O_31,6.0,89.0,0.0,45.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
596,TRAIN_596,1,0.531375,2022-09-08 14:38,T100304,O_31,40.0,94.0,0.0,45.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
597,TRAIN_597,1,0.533702,2022-09-08 14:46,T100306,O_31,21.0,87.0,0.0,45.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [90]:
train_df = pd.concat([train_dfaf, train_dftf, train_dfof], axis = 0)
train_df = train_df.sort_index(ascending=True)

### Feature Selection

outlier 이상체 제거한 최종 데이터 프레임 저장하기 & 모든 value가 결측치이거나 0.0인 행 지운 test data도 저장하기

In [91]:
train_df.to_csv('./train_fillnan_t28_med.csv', index = False) 

In [154]:
# test_df.to_csv('./test_fillnan_t23.csv', index = False) #traindf의 수치데이터를 testdf에 다시 적용해 전처리

모델링을 하기 앞서, 독립변수 X와 종속변수 y를 설정해야한다.

**학습에 사용할 변수 X와 예측할 변수 y를 분리**

In [92]:
train_y = train_df['Y_Class']

학습에 쓰이지 않을 column들을 제거한다.

In [93]:
train_x = train_df.drop(columns = ['PRODUCT_ID', 'TIMESTAMP', 'Y_Class', 'Y_Quality'])
#모델 학습이 끝나고 예측에 쓰일 test데이터
test_x = test_df.drop(columns=['PRODUCT_ID', 'TIMESTAMP'])

범주형 데이터를 수치 데이터로 전환하기 위해 LabelEncoder 활용

In [94]:
# qualitative to quantitative
qual_col = ['LINE', 'PRODUCT_CODE']

for i in qual_col:
    le = LabelEncoder()
    le = le.fit(train_x[i]) #원래 column 값을 기준으로 fit.
    train_x[i] = le.transform(train_x[i]) #수치화, 수치로 변형

    for label in np.unique(test_x[i]): 
        if label not in le.classes_: 
            le.classes_ = np.append(le.classes_, label)
    test_x[i] = le.transform(test_x[i]) 
print('Done.')

Done.


In [112]:
test_x.isnull().sum()[test_x.isnull().sum() > 0]

X_2628    12
X_2629    12
X_2630    12
X_2631    12
X_2632    12
          ..
X_2696    12
X_2697    12
X_2698    12
X_2699    12
X_2844    12
Length: 73, dtype: int64

### Modeling

1.모델 선택 - sklearn라이브러리 활용 - RandomForest 

2.모델 학습 - train_df를 활용하여 1번에서 정의한 모델로 학습

3.예측 - 학습된 모델을 바탕으로 test 데이터를 예측

4.정답 파일 생성 - 정답 파일 생성 및 제출 필요(경진대회를 위해 필요한 과정.)

subRF_r = pd.read_csv('./sample_submission.csv')
subXGB_r = pd.read_csv('./sample_submission.csv')
sub_v = pd.read_csv('./sample_submission.csv')

# Randomsearch

1.XGB randomsearch

### XGB randomsearch

In [100]:
param_distribs = {
#     'eta' : randint(low = 0.01, high = 0.1),
    'n_estimators': randint(low = 30, high = 100),
    'min_child_weight': randint(low = 1, high = 10),
    'gamma': randint(low = 1, high = 10),
    'max_depth': randint(low = 3, high = 8),
    'lambda' : randint(low = 1, high = 10),
    'alpha' : randint(low = 1, high = 10)
}
start = time.time()
xgboost_r = XGBClassifier(seed=37).fit(train_x,train_y)
rand_cv = RandomizedSearchCV(xgboost_r, 
                            param_distributions=param_distribs,
                            cv = 5, 
                            n_iter = 50,
                            scoring = 'f1_macro',
                            n_jobs = -1,
                            verbose=3)
rand_cv.fit(train_x, train_y)
preds = rand_cv.predict(test_x)
end = time.time()
sub5['Y_Class'] = preds
sub5.to_csv('./t28_XGB_random_redata_missingvalue_processing_forsome_med_addparam.csv', index=False)

print(f'최적 하이퍼 파라미터: {rand_cv.best_params_}')
print(f'최고 예측 정확도: {(rand_cv.best_score_)*100:.4f}')
print(f'time elapsed: {end-start}')

Fitting 5 folds for each of 50 candidates, totalling 250 fits
최적 하이퍼 파라미터: {'alpha': 3, 'gamma': 2, 'lambda': 6, 'max_depth': 5, 'min_child_weight': 5, 'n_estimators': 60}
최고 예측 정확도: 85.3181
time elapsed: 793.5489270687103


In [70]:
sub1.describe() #mean fill 처리 그냥 randomsearch로

,Y_Class
count,310.000000
mean,1.009677
std,0.620499
min,0.000000
25%,1.000000
50%,1.000000
75%,1.000000
max,2.000000


In [72]:
xgboost_r_p = XGBClassifier(seed=37, alpha = 8, gamma = 1, max_depth = 5, min_child_weight = 5).fit(train_x,train_y)
preds = xgboost_r_p.predict(test_x)
sub2['Y_Class'] = preds
sub2.to_csv('./t28_XGB_random_redata_missingvalue_processing_forsome_p.csv', index=False)

In [74]:
sub2.describe()# randomsearch로 찾은 최적의 파라미터를 다시 그냥 XGB 모델에 넣기

,Y_Class
count,310.000000
mean,1.316129
std,0.571772
min,0.000000
25%,1.000000
50%,1.000000
75%,2.000000
max,2.000000


In [75]:
xgboost_r_d = XGBClassifier(seed=37).fit(train_x, train_y)
preds = xgboost_r_d.predict(test_x)
sub3['Y_Class'] = preds
sub3.to_csv('./t28_XGB_random_redata_missingvalue_processing_forsome_pd.csv', index=False)

In [76]:
sub3.describe() #그냥 디폴트 파라미터로 XGB에 돌리기

,Y_Class
count,310.000000
mean,0.809677
std,0.454289
min,0.000000
25%,1.000000
50%,1.000000
75%,1.000000
max,2.000000


In [96]:
sub4.describe()

,Y_Class
count,310.000000
mean,1.193548
std,0.639547
min,0.000000
25%,1.000000
50%,1.000000
75%,2.000000
max,2.000000


[CV 2/5] END alpha=6, gamma=7, lambda=4, max_depth=8, min_child_weight=6;, score=0.701 total time=  45.7s
[CV 3/5] END alpha=4, gamma=3, lambda=9, max_depth=9, min_child_weight=2;, score=0.988 total time=  54.6s
[CV 1/5] END alpha=4, gamma=3, lambda=4, max_depth=9, min_child_weight=4;, score=0.604 total time=  36.9s
[CV 2/5] END alpha=3, gamma=4, lambda=8, max_depth=6, min_child_weight=4;, score=0.722 total time=  40.7s
[CV 5/5] END alpha=5, gamma=2, lambda=4, max_depth=8, min_child_weight=5;, score=0.968 total time=  41.3s
[CV 4/5] END alpha=7, gamma=6, lambda=9, max_depth=8, min_child_weight=7;, score=0.908 total time=  45.0s
[CV 4/5] END alpha=1, gamma=8, lambda=9, max_depth=5, min_child_weight=2;, score=0.908 total time=  43.8s
[CV 4/5] END alpha=3, gamma=5, lambda=6, max_depth=5, min_child_weight=8;, score=0.933 total time=  36.1s
[CV 2/5] END alpha=3, gamma=5, lambda=8, max_depth=6, min_child_weight=2;, score=0.679 total time=  50.3s
[CV 1/5] END alpha=7, gamma=7, lambda=4, max_d

[CV 3/5] END alpha=9, gamma=6, lambda=8, max_depth=5, min_child_weight=9;, score=0.957 total time=  36.0s
[CV 1/5] END alpha=4, gamma=3, lambda=9, max_depth=9, min_child_weight=2;, score=0.591 total time=  46.7s
[CV 4/5] END alpha=9, gamma=4, lambda=9, max_depth=9, min_child_weight=2;, score=0.945 total time=  57.7s
[CV 3/5] END alpha=3, gamma=4, lambda=8, max_depth=6, min_child_weight=4;, score=0.988 total time=  41.6s
[CV 1/5] END alpha=3, gamma=5, lambda=5, max_depth=5, min_child_weight=5;, score=0.579 total time=  33.6s
[CV 3/5] END alpha=7, gamma=6, lambda=9, max_depth=8, min_child_weight=7;, score=0.990 total time=  41.9s
[CV 2/5] END alpha=1, gamma=8, lambda=9, max_depth=5, min_child_weight=2;, score=0.627 total time=  42.3s
[CV 1/5] END alpha=3, gamma=5, lambda=6, max_depth=5, min_child_weight=8;, score=0.597 total time=  30.7s
[CV 4/5] END alpha=8, gamma=5, lambda=2, max_depth=9, min_child_weight=5;, score=0.933 total time=  48.8s
[CV 2/5] END alpha=2, gamma=8, lambda=1, max_d

[CV 4/5] END alpha=6, gamma=7, lambda=4, max_depth=8, min_child_weight=6;, score=0.908 total time=  53.8s
[CV 1/5] END alpha=9, gamma=4, lambda=9, max_depth=9, min_child_weight=2;, score=0.581 total time=  46.8s
[CV 2/5] END alpha=4, gamma=3, lambda=4, max_depth=9, min_child_weight=4;, score=0.709 total time=  42.4s
[CV 4/5] END alpha=3, gamma=4, lambda=8, max_depth=6, min_child_weight=4;, score=0.933 total time=  42.4s
[CV 3/5] END alpha=3, gamma=5, lambda=5, max_depth=5, min_child_weight=5;, score=0.988 total time=  37.3s
[CV 1/5] END alpha=4, gamma=2, lambda=3, max_depth=9, min_child_weight=5;, score=0.650 total time=  31.7s
[CV 1/5] END alpha=1, gamma=8, lambda=9, max_depth=5, min_child_weight=2;, score=0.582 total time=  37.9s
[CV 4/5] END alpha=8, gamma=1, lambda=8, max_depth=5, min_child_weight=5;, score=0.957 total time=  35.8s
[CV 2/5] END alpha=8, gamma=5, lambda=2, max_depth=9, min_child_weight=5;, score=0.709 total time=  45.5s
[CV 5/5] END alpha=3, gamma=5, lambda=8, max_d

In [101]:
sub5.describe() #addparam

,Y_Class
count,310.000000
mean,1.354839
std,0.560189
min,0.000000
25%,1.000000
50%,1.000000
75%,2.000000
max,2.000000


[CV 1/5] END alpha=2, gamma=1, lambda=5, max_depth=6, min_child_weight=7, n_estimators=35;, score=0.556 total time=   8.9s
[CV 4/5] END alpha=2, gamma=1, lambda=5, max_depth=6, min_child_weight=7, n_estimators=35;, score=0.957 total time=  11.7s
[CV 2/5] END alpha=8, gamma=4, lambda=6, max_depth=5, min_child_weight=2, n_estimators=96;, score=0.709 total time=  38.0s
[CV 5/5] END alpha=2, gamma=6, lambda=2, max_depth=7, min_child_weight=5, n_estimators=90;, score=0.906 total time=  40.7s
[CV 4/5] END alpha=2, gamma=4, lambda=8, max_depth=3, min_child_weight=2, n_estimators=97;, score=0.945 total time=  26.6s
[CV 3/5] END alpha=9, gamma=2, lambda=3, max_depth=5, min_child_weight=4, n_estimators=30;, score=1.000 total time=  11.3s
[CV 1/5] END alpha=7, gamma=5, lambda=3, max_depth=7, min_child_weight=2, n_estimators=81;, score=0.583 total time=  37.2s
[CV 4/5] END alpha=6, gamma=7, lambda=8, max_depth=4, min_child_weight=9, n_estimators=40;, score=0.933 total time=  12.6s
[CV 5/5] END alp

[CV 1/5] END alpha=4, gamma=8, lambda=5, max_depth=6, min_child_weight=5, n_estimators=69;, score=0.574 total time=  25.2s
[CV 4/5] END alpha=8, gamma=4, lambda=6, max_depth=5, min_child_weight=2, n_estimators=96;, score=0.933 total time=  40.0s
[CV 2/5] END alpha=2, gamma=8, lambda=1, max_depth=6, min_child_weight=5, n_estimators=98;, score=0.725 total time=  40.3s
[CV 5/5] END alpha=2, gamma=4, lambda=8, max_depth=3, min_child_weight=2, n_estimators=97;, score=0.957 total time=  26.2s
[CV 2/5] END alpha=7, gamma=5, lambda=6, max_depth=6, min_child_weight=1, n_estimators=76;, score=0.679 total time=  38.3s
[CV 1/5] END alpha=6, gamma=7, lambda=8, max_depth=4, min_child_weight=9, n_estimators=40;, score=0.571 total time=  11.1s
[CV 1/5] END alpha=6, gamma=5, lambda=6, max_depth=6, min_child_weight=6, n_estimators=43;, score=0.604 total time=  14.7s
[CV 2/5] END alpha=5, gamma=6, lambda=3, max_depth=4, min_child_weight=4, n_estimators=61;, score=0.735 total time=  19.9s
[CV 5/5] END alp

[CV 2/5] END alpha=4, gamma=8, lambda=5, max_depth=6, min_child_weight=5, n_estimators=69;, score=0.725 total time=  28.7s
[CV 5/5] END alpha=8, gamma=4, lambda=6, max_depth=5, min_child_weight=2, n_estimators=96;, score=0.931 total time=  39.1s
[CV 3/5] END alpha=2, gamma=8, lambda=1, max_depth=6, min_child_weight=5, n_estimators=98;, score=0.988 total time=  40.4s
[CV 1/5] END alpha=3, gamma=2, lambda=8, max_depth=6, min_child_weight=9, n_estimators=46;, score=0.559 total time=  11.4s
[CV 5/5] END alpha=3, gamma=2, lambda=8, max_depth=6, min_child_weight=9, n_estimators=46;, score=0.968 total time=  14.2s
[CV 4/5] END alpha=7, gamma=5, lambda=6, max_depth=6, min_child_weight=1, n_estimators=76;, score=0.945 total time=  37.9s
[CV 2/5] END alpha=6, gamma=7, lambda=8, max_depth=4, min_child_weight=9, n_estimators=40;, score=0.732 total time=  12.6s
[CV 3/5] END alpha=6, gamma=5, lambda=6, max_depth=6, min_child_weight=6, n_estimators=43;, score=0.978 total time=  17.7s
[CV 4/5] END alp

In [1]:
fig, ax = plt.subplots(figsize=(10, 100))
plot_importance(xgboost_r, ax=ax)


NameError: name 'plt' is not defined

In [11]:
sub1.describe()

,Y_Class
count,310.000000
mean,0.883871
std,0.340471
min,0.000000
25%,1.000000
50%,1.000000
75%,1.000000
max,2.000000


In [12]:
sub3 = pd.read_csv('/Users/kimminyoung/Desktop/Dacon_SmartFactory/t10_GBM_RandomSearchCV_niter50(0.640).csv')
sub3.describe()

,Y_Class
count,310.000000
mean,0.880645
std,0.592793
min,0.000000
25%,1.000000
50%,1.000000
75%,1.000000
max,2.000000


In [16]:
sub4 = pd.read_csv('./t22_sep_data_com(0.592).csv')

In [17]:
sub = pd.concat([sub1, sub3, sub4], axis = 1)
sub.iloc[0:20,:]

,PRODUCT_ID,Y_Class,PRODUCT_ID,Y_Class,PRODUCT_ID,Y_Class
0,TEST_000,1,TEST_000,1,TEST_000,1
1,TEST_001,1,TEST_001,1,TEST_001,1
2,TEST_002,1,TEST_002,1,TEST_002,1
3,TEST_003,1,TEST_003,1,TEST_003,1
4,TEST_004,1,TEST_004,0,TEST_004,1
5,TEST_005,1,TEST_005,1,TEST_005,1
6,TEST_006,1,TEST_006,1,TEST_006,0
7,TEST_007,0,TEST_007,0,TEST_007,0
8,TEST_008,1,TEST_008,0,TEST_008,0
9,TEST_009,1,TEST_009,1,TEST_009,1


In [18]:
# #train_df_0 index 재설정
# train_df_0 = train_df_0.reset_index(inplace = False, drop = True)
# train_df_0

,PRODUCT_ID,Y_Class,Y_Quality,TIMESTAMP,LINE,PRODUCT_CODE,X_1,X_2,X_3,X_4,...,X_2866,X_2867,X_2868,X_2869,X_2870,X_2871,X_2872,X_2873,X_2874,X_2875
0,TRAIN_022,0,0.517719,2022-06-14 8:53,T100304,T_31,2.0,102.0,0.0,45.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,TRAIN_023,0,0.519090,2022-06-14 9:01,T100304,T_31,2.0,102.0,0.0,45.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,TRAIN_028,0,0.521249,2022-06-19 20:26,T010305,A_31,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,TRAIN_034,0,0.521524,2022-06-21 17:36,T050304,A_31,NaN,NaN,NaN,NaN,...,55.03,52.24,55.33,57.49,67.31,1.0,NaN,NaN,NaN,NaN
4,TRAIN_066,0,0.524408,2022-06-25 21:38,T010305,A_31,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83,TRAIN_583,0,0.522340,2022-09-05 8:34,T050304,A_31,NaN,NaN,NaN,NaN,...,51.71,59.64,54.61,57.05,63.18,1.0,NaN,NaN,NaN,NaN
84,TRAIN_584,0,0.519519,2022-09-05 11:09,T010305,A_31,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
85,TRAIN_585,0,0.515214,2022-09-05 11:17,T010306,A_31,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
86,TRAIN_594,0,0.524022,2022-09-08 22:38,T050304,A_31,NaN,NaN,NaN,NaN,...,49.47,53.07,50.89,55.10,66.49,1.0,NaN,NaN,NaN,NaN
